In [1]:
! ls

CV_function.ipynb         Data_Preprocessing.ipynb  pram_grid.ipynb
Data_EDA_First_Week.ipynb main.ipynb


In [2]:
# Standard library imports
import itertools
import json
import datetime
import pathlib

# Third-party imports
import pandas as pd
import torch
from sklearn.model_selection import ShuffleSplit

# Slim-GSGP imports
from slim_gsgp.datasets.data_loader import load_pandas_df  
from slim_gsgp.utils.utils import train_test_split  
from slim_gsgp.main_gp import gp
#from slim_gsgp.main_gsgp import gsgp
#from slim_gsgp.main_slim import slim
from slim_gsgp.evaluators.fitness_functions import rmse

## Load Data 

In [3]:
# Reading the desired dataset
df = pd.read_csv("../data/sustavianfeed.csv", sep=';')

# Dropping the first column (index) and renaming the columns
df = df.drop(columns= ['WING TAG', 'EMPTY MUSCULAR STOMACH'])

# Moving crude protein to the end of the dataframe
df = df[[col for col in df.columns if col != 'CRUDE PROTEIN'] + ['CRUDE PROTEIN']] 

In [4]:
# Turning df into X and y torch.Tensors
X, y = load_pandas_df(df, X_y=True)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)

# Split the test set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)

# grid search

In [5]:
import os
os.chdir(os.path.join(os.getcwd(), os.pardir))

In [6]:
from utils.grid_search_v2_leo import grid_search



# Define a parameter grid for the grid search
param_grid = {
     "fit_intercept": [True, False]
}

# Perform grid search
grid_search(
    X_train,
    y_train,
    param_grid=param_grid
)


Grid Search Progress: 100%|██████████| 2/2 [00:00<00:00, 186.20combination/s]


({'fit_intercept': False},
 -46.341706085205075,
                model  fit_intercept  mean_score  \
 1  linear regression          False  -46.341706   
 0  linear regression           True    0.026605   
 
                                          fold_scores  \
 1  [-70.49513244628906, -53.06315612792969, -9.38...   
 0  [-0.20772814750671387, -0.31918060779571533, 0...   
 
                     timestamp  
 1  2025-05-12T12:37:05.293858  
 0  2025-05-12T12:37:05.291381  )

# Visualizations

In [471]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import ast  
from collections import defaultdict

In [461]:
df = pd.read_csv("../log/gp.csv", header=None)
settings_df = pd.read_csv("../log/gp_settings.csv", header=None)
unique_settings_df = settings_df.drop_duplicates(0) #the comb id is now indexed

In [462]:
unique_settings_df.head()

,0,1
0,865732c8-3014-11f0-b37b-baa0ecd080fe,"{'log': 1, 'verbose': 0, 'test_elite': True, '..."
1,8f386164-3014-11f0-b37b-baa0ecd080fe,"{'log': 1, 'verbose': 0, 'test_elite': True, '..."
2,9498ad58-3014-11f0-b37b-baa0ecd080fe,"{'log': 1, 'verbose': 0, 'test_elite': True, '..."
3,9cac4a04-3014-11f0-b37b-baa0ecd080fe,"{'log': 1, 'verbose': 0, 'test_elite': True, '..."
4,a37fb898-3014-11f0-b37b-baa0ecd080fe,"{'log': 1, 'verbose': 0, 'test_elite': True, '..."


In [ ]:
# 0  - Algorithm
# 1  - Instance ID
# 2  - Dataset
# 3  - Seed
# 4  - Generation
# 5  - Fitness
# 6  - Running time
# 7  - Population nodes
# 8  - Test fitness
# 9  - Elite nodes
# 10 - niche entropy
"""From here on, it doesnt appear on df"""
# 11 - sd(pop.fit)
# 12 - Log level 

In [ ]:
'''def get_combination_str(setting_str, unique_settings_df = pd.DataFrame(df[1].unique())):
    comb_str = unique_settings_df[unique_settings_df[0]==setting_str][1][0]
    return comb_str'''

def param_in_combination(param:str, comb_str: str):
    """
    Parameters that the function can receive:
    'log' / 'verbose'/ 'test_elite' / 'n_jobs' / 'max_depth' / 'n_elites' / 'elistism' / 'n_iter' 
    'settings_dict' / 'p_xo' / 'pop_size' / 'seed' / 'p_m' / 'p_c' / 'init_depth' / 'init_pop_size'
    """
    variable_init = comb_str.find(param) 
    param_init = variable_init + len(param) + 3 #advance 3 steps to account for the quote, the dots and the space. 
    param_end = comb_str.find(',', variable_init)
    param_value = comb_str[param_init:param_end]
    if param_value.startswith('<') or param_value.startswith('['):
        print('The value of the parameter given cannot be converted to its original class:')
    else:
        try:
            return ast.literal_eval(param_value)
        except SyntaxError as s:
            print(f'Parameter does not exist in the combination string or it cannot be accessed. ({s})')

#example
#print(get_combination_str('865732c8-3014-11f0-b37b-baa0ecd080fe'))
#print(param_in_combination('log', get_combination_str('865732c8-3014-11f0-b37b-baa0ecd080fe')))
    
    

In [ ]:
'''def pop_fitness_diversity(df, train_color='blue'):
     """
     Out of Bounds
     """
     dif_combs = np.unique(df[[1]])
     for comb in dif_combs:
          y = df[df[1]==comb]
          #comb_dict = get_combination(comb)
          fig = go.Figure()
          fig.add_trace(go.Scatter(y=y.iloc[:,11].values, 
                                   mode='lines', name='Train', line=dict(color=train_color)))
          fig.update_layout(
          height=400, width=800, 
          margin=dict(t=50),
          yaxis_range=[0,None],
          title_text=f'GP - Population Fitness Diversity\nCombination:',
          xaxis_title='Generation', yaxis_title='Fitness Standard Deviation'
          )
          fig.show()'''



In [463]:
def train_test_fit(df, train_color='blue', test_color='orange', rows=5, cols=4):
    dif_combs = df[1].unique()  # Get unique combinations
    unique_setting_df = pd.DataFrame(dif_combs)
    num_plots = len(dif_combs)
    assert rows*cols==num_plots
    
    # Create subplot grid
    fig = sp.make_subplots(rows=rows, cols=cols, 
                           subplot_titles=[f"Combination index: {unique_settings_df[unique_settings_df[0]==comb].index[0]}" 
                                           for comb in dif_combs])
    
    for i, comb in enumerate(dif_combs):
        y = df[df[1] == comb]
        algo = y.iloc[0,0]
        row = (i // cols) + 1  #Calculate row position
        col = (i % cols) + 1   #Calculate column position
        
        fig.add_trace(
            go.Scatter(y=y.iloc[:, 5].values, mode='lines', name='Train', line=dict(color=train_color),
                       showlegend=(i==0)),
            row=row, col=col
        )
        
        fig.add_trace(
            go.Scatter(y=y.iloc[:, 8].values, mode='lines', name='Test', line=dict(color=test_color),
                       showlegend=(i==0)),
            row=row, col=col
        )

        fig.update_yaxes(range=[0, None], row=row, col=col)
    
    fig.update_layout(
        height=150 * rows,
        width=250 * cols,
        margin=dict(t=50),
        title_text=f'{algo} - Train vs Test Fitness (x=Generation, y=RMSE)',
        showlegend=True
    )

    fig.update_annotations(font_size=10)
    fig.show()

In [464]:
def train_test_fit_and_size(df, comb_idxs: list | int = [i for i in range(pd.DataFrame(df[1].unique()).shape[0])],
                            train_color='blue', test_color='orange'):
     unique_setting_df = pd.DataFrame(df[1].unique())
     for comb_idx in comb_idxs:
          comb = unique_settings_df.iloc[comb_idx, 0]
          y = df[df[1]==comb]
          algo = y.iloc[0,0]
          fig = make_subplots(
          rows=1, cols=2,
          subplot_titles=(f'{algo} - Fitness evolution\nCombination:', f'{algo} - Size evolution')
          )

          fig.add_trace(go.Scatter(y=y.iloc[:,5].values, 
                                   mode='lines', name='Train', line=dict(color=train_color)), row=1, col=1)
          fig.add_trace(go.Scatter(y=y.iloc[:,8].values, 
                                   mode='lines', name='Test', line=dict(color=test_color)), row=1, col=1)
          fig.add_trace(go.Scatter(y=y.iloc[:,9].values, 
                                   mode='lines', name='Size'), row=1, col=2)
          
          fig.update_xaxes(title_text="Generation")

          fig.update_layout(
          width=1000,
          height=400, 
          showlegend=True,
          yaxis_range=[0,None],
          )
          fig.show()

In [465]:
def niche_entropy(df, train_color='blue', rows=5, cols=4):
    dif_combs = df[1].unique()  # Get unique combinations
    unique_setting_df = pd.DataFrame(dif_combs) # array to df
    num_plots = len(dif_combs)
    assert rows*cols==num_plots, "The number of combinations does not correspond to the grid size defined (rows/cols)."

    fig = sp.make_subplots(rows=rows, cols=cols, 
                           subplot_titles=[f"Combination index: {unique_settings_df[unique_settings_df[0]==comb].index[0]}" 
                                           for comb in dif_combs])

    for i, comb in enumerate(dif_combs):
        y = df[df[1] == comb]
        algo = y.iloc[0,0]
        row = (i // cols) + 1
        col = (i % cols) + 1
        
        fig.add_trace(
            go.Scatter(
                y=y.iloc[:, 10].values,
                mode='lines',
                name='Niche Entropy',
                line=dict(color=train_color),
                showlegend=(i == 0)), row=row, col=col
                )
    
    fig.update_layout(
        height=150 * rows,
        width=250 * cols,
        margin=dict(t=50),
        title_text=f'{algo} - Niche Entropy (x=Generation, y=Entropy)',
    )
    
    fig.show()

In [466]:
def plot_combs_together_test(df, comb_idxs: list | int = [i for i in range(pd.DataFrame(df[1].unique()).shape[0])],
                             colors = ['#FF0000', '#0000FF', '#00FF00', '#FFA500', '#800080', 
                                       '#FF00FF', '#00FFFF', '#FFFF00', '#1F77B4', '#FF7F0E',
                                       '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2',
                                       '#7F7F7F', '#AEC7E8', '#FFBB78', '#98DF8A', '#FF9896'],
                              ):
     
     assert len(colors)>=len(comb_idxs), "Not enough colors for all combinations"

     unique_settings_df = pd.DataFrame(df[1].unique())
     fig = go.Figure()
     for i, comb_idx in enumerate(comb_idxs):
          comb = unique_settings_df.iloc[comb_idx, 0]
          y = df[df[1]==comb]
          algo = y.iloc[0,0]

          fig.add_trace(go.Scatter(y=y.iloc[:,8].values, 
                                   mode='lines', name=f'Test Comb {comb_idx}',
                                   line=dict(color=colors[i])))#, row=1, col=1)
          
          fig.update_xaxes(title_text="Generation")

     fig.update_layout(
          width=1000,
          height=400, 
          title_text = f"{algo} - Test Fitness (Combinations indexes: {comb_idxs})",
          showlegend=True,
          yaxis_range=[0,None],
          )
     
     fig.show()



In [ ]:
train_test_fit(df)

In [ ]:
train_test_fit_and_size(df, comb_idxs=[3,19])

In [ ]:
niche_entropy(df)

In [ ]:
plot_combs_together_test(df,comb_idxs=[1,2,3])

Modular functions for different versions/hyperparameters combinations
ASSUMING THAT:
- 

In [ ]:
SLIM_VERSIONS = ['SLIM+SIG2', 'SLIM+SIG1', 'SLIM+ABS', 'SLIM*SIG2', 'SLIM*SIG1', 'SLIM*ABS']
COMBINATIONS = [ for i in df[1].unique()]

"""param_grid = {
    'slim_version': SLIM_VERSIONS
}"""

In [ ]:
rmse_by_config = defaultdict(list)

for split in results:
    rmse_train = []
    rmse_test = []
    
    for result in split:
        key = ''
        for k, v in result['dynamic_params'].items():
            key += k+': '+str(v)+' <br /> '
        rmse_by_config[key].append(result['rmse_test'])

fig = go.Figure()
for config, rmse_values in rmse_by_config.items():
    fig.add_trace(go.Box(
        y=rmse_values,
        boxpoints='all',
        jitter=0.5,
        pointpos=0,
        line=dict(color='orange'),
        name=config
    ))

fig.update_layout(
    title=DATASET_NAME+' dataset',
    xaxis_title='',
    yaxis_title='Test RMSE',
    height=500, width=1100,
    xaxis_tickangle=-90,
    yaxis_range=[0,None],
    margin=dict(l=50, r=50, t=50, b=20),
    showlegend=False,
    template='plotly_white'
)

fig.show()

In [ ]:
# Plot settings
df_log = []
for i_inner in range(k_inner):
    tmp = pd.read_csv(LOG_DIR+'slim_'+DATASET_NAME+'_'+str(i_inner)+'.csv', header=None)
    tmp['cv'] = i_inner
    df_log.append(tmp)
df_log = pd.concat(df_log, ignore_index=True)

n_rows = 2
n_cols = 3

In [ ]:
make_evolution_plots(n_rows, n_cols, SLIM_VERSIONS, df_log,
                     plot_title = 'SLIM - Train vs Test Fitness ('+DATASET_NAME+' dataset)')
[fixed_params['pop_size'], fixed_params['tournament_size']]

In [ ]:
make_evolution_plots(n_rows, n_cols, SLIM_VERSIONS, df_log, var='size'
                     plot_title = 'SLIM -Size ('+DATASET_NAME+' dataset)')